# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner

In [11]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [12]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/3-5-models-and-computer-use',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosure-policy',
 'https://trust.anthropic.com/']

In [13]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://anthropic.com/team'},
  {'type': 'research page', 'url': 'https://anthropic.com/research'},
  {'type': 'product page', 'url': 'https://anthropic.com/claude'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://anthropic.com"))

In [25]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [54]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += f"The brochure should have a Spanish translation inline after every section.  It should be in Italics.\n"
    user_prompt += f"Add a section at the end called Our Humor and provide a corny joke or funny short story - no more than a paragraph"
    user_prompt += get_all_details(url)
  
    
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [55]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}]}


"You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nThe brochure should have a Spanish translation inline after every section.  It should be in Italics.\nAdd a section at the end called Our Humor and provide a corny joke or funny short story - no more than a paragraphLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nStart using Claude to drive efficiency and create new revenue streams.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work

In [56]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [57]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}]}


# Welcome to Anthropic!
*¡Bienvenido a Anthropic!*

**Where AI Safety Takes the Spotlight!**  
*Donde la seguridad de la IA toma el centro del escenario!*

---

### Meet Claude!
*¡Conoce a Claude!*

You’ve heard about ChatGPT, now let’s meet Claude! Our chatty AI partner, Claude 3.5 Sonnet, is not just smart—he's also great at poetry! With Claude at your side, your emails will flow like Shakespearean sonnets (or at least they'll be typo-free)!  
*(Has oído hablar de ChatGPT, ahora conozcamos a Claude. ¡Nuestro compañero de IA parlante, Claude 3.5 Sonnet, no solo es inteligente, también es genial con la poesía! ¡Con Claude a tu lado, tus correos electrónicos fluirán como sonetos de Shakespeare (o al menos estarán sin errores tipográficos)!)*

---

### Our Purpose: AI for Good!
*Nuestro Propósito: IA para el Bien!*

At Anthropic, we believe that AI should build a better world, not a robot apocalypse (sorry, Terminator fans). We’re committed to harnessing AI safely so that society flourishes, rather than flounders. Here’s to a future where your toaster does NOT try to take over the world!  
*(En Anthropic, creemos que la IA debe construir un mundo mejor, no un apocalipsis de robots (lo siento, fanáticos de Terminator). Estamos comprometidos a aprovechar la IA de forma segura para que la sociedad florezca, en lugar de hundirse. ¡Brindemos por un futuro donde tu tostadora NO intentará apoderarse del mundo!)*

---

### Company Culture: Trust, Teamwork & Simplicity
*Cultura de la Empresa: Confianza, Trabajo en Equipo y Simplicidad*

Our culture is all about high trust—like that friend who “borrows” your last piece of pizza and promises to replace it (but we really do mean it!). Collaboration is key, and we believe in doing the simple things that work—like asking Claude for help before tackling that complicated report.  
*(Nuestra cultura se basa en una alta confianza; como ese amigo que "toma prestada" tu última pieza de pizza y promete reemplazarla (¡pero realmente lo decimos!). La colaboración es fundamental, y creemos en hacer las cosas simples que funcionan, como pedirle ayuda a Claude antes de abordar ese informe complicado.)*

---

### Customers: A Variety of Big Thinkers
*Clientes: Una Variedad de Grandes Pensadores*

From startups to NGOs, our clients are seriously smart folks who want to leverage Claude's brilliance to enhance efficiency. Whether you run a small skateboard company or an academic institution, Claude is ready to roll!  
*(Desde startups hasta ONG, nuestros clientes son personas realmente inteligentes que desean aprovechar la brillantez de Claude para mejorar la eficiencia. ¡Ya sea que dirijas una pequeña compañía de patinetas o una institución académica, Claude está listo para rodar!)*

---

### Careers at Anthropic: Join the Adventure!
*Carreras en Anthropic: ¡Únete a la Aventura!*

Love AI? So do we! Join us at Anthropic, where we blend research, engineering, and a sprinkle of horticulture appreciation. We offer competitive salaries, wellness perks, and most importantly, an atmosphere where you can actually have a healthy work-life balance (yes, pizza breaks are encouraged!).  
*(¿Amas la IA? ¡Nosotros también! Únete a nosotros en Anthropic, donde combinamos investigación, ingeniería y un toque de aprecio por la horticultura. Ofrecemos salarios competitivos, beneficios de bienestar y, lo más importante, un ambiente donde realmente puedes tener un equilibrio saludable entre el trabajo y la vida (sí, ¡las pausas para la pizza son bienvenidas!).)*

---

## Our Humor: 
*Nuestro Humor:*

Why did the AI cross the road?  
To optimize the algorithm on the other side, of course!   
*(¿Por qué cruzó la IA la carretera? Para optimizar el algoritmo del otro lado, ¡por supuesto!)*

---

So, whether you're a customer, investor, or a future colleague, remember: at Anthropic, we're not just building AI—we're building a safer future, one Claude at a time!  
*Así que, ya seas un cliente, inversor o colega futuro, recuerda: en Anthropic, no solo estamos construyendo IA, ¡estamos construyendo un futuro más seguro, un Claude a la vez!*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [58]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [59]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


# Welcome to Anthropic! 🌟 
*¡Bienvenido a Anthropic!*

At Anthropic, we’re on a mission that’s actually worth talking about—developing AI systems that are safe, reliable, and something you can trust with your coffee order! ☕🤖  
*En Anthropic, estamos en una misión que vale la pena hablar: desarrollar sistemas de IA que sean seguros, confiables y en los que puedas confiar con tu pedido de café!*

---

## Meet Claude (No, Not That One) 👤
*Conoce a Claude (no, no ese)*

Say hello to Claude 3.5 Sonnet—our most intelligent AI model. He’s not just a fancy virtual assistant; he’s like that one friend who always brings snacks and helps you brainstorm! 🎉  
*Di hola a Claude 3.5 Sonnet, nuestro modelo de IA más inteligente. No es solo un asistente virtual elegante, ¡es como ese amigo que siempre trae bocadillos y te ayuda a hacer una lluvia de ideas!*

---

## Our Company Culture 🤝
*Nuestra cultura empresarial*

At Anthropic, trust isn't just a word; it's our lifestyle. We assume good faith, prioritize honesty, and disagree like polite adults. Plus, we have a soft spot for plants! 🌱  So if you're looking for a workplace where you can pet a fern as you contemplate AI safety, look no further!  
*En Anthropic, la confianza no es solo una palabra; es nuestro estilo de vida. Asumimos buena fe, priorizamos la honestidad y discrepamos como adultos educados. ¡Además, tenemos un amor por las plantas! Así que, si buscas un lugar de trabajo donde puedas acariciar un helecho mientras contemplas la seguridad de la IA, ¡no busques más!*

---

## Join Our Team! 🚀
*¡Únete a nuestro equipo!*

Think you can make AI safer AND have fun while doing it? We want you! We're looking for talented people in research, engineering, policy, and even those who just have a knack for making great coffee. If you can contribute to our mission, you belong here!  
*¿Crees que puedes hacer que la IA sea más segura Y divertirte mientras lo haces? ¡Te queremos! Buscamos personas talentosas en investigación, ingeniería, políticas e incluso aquellas que simplemente tengan un talento para hacer buen café. Si puedes contribuir a nuestra misión, perteneces aquí!*

---

## Our Customers: Usually Not Villains 😎
*Nuestros clientes: generalmente no son villanos*

We cater to a variety of industries, from nonprofits to big enterprises, and our only villain is not providing value. So whether you're a small startup or a large organization, Claude is here to save your day! 🦸‍♂️  
*Atendemos a una variedad de industrias, desde organizaciones sin fines de lucro hasta grandes empresas, y nuestro único villano es no brindar valor. Así que, ya seas una pequeña startup o una gran organización, ¡Claude está aquí para salvar tu día!*

---

## Our Humor 😄
*Nuestro humor*

Why did the AI go to therapy? Because it had too many neural issues! 🤖💔  
*¿Por qué fue la IA a terapia? ¡Porque tenía demasiados problemas neuronales!*

--- 

So remember, whether you're a customer looking for reliable AI or a potential recruit who loves plants and trust (and probably coffee), Anthropic has a place for you!  
*Así que recuerda, ya seas un cliente en busca de IA confiable o un recluta potencial que ama las plantas y la confianza (y probablemente el café), ¡Anthropic tiene un lugar para ti!*

In [60]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face: Your Friendly AI Playground!

Welcome to Hugging Face – the AI community that's not just building the future but is also shoving cupcakes in its mouth while doing it! We’re here to make machine learning as delightful as a warm hug and to sweeten the deal, we’ve got something for everyone – from curious customers to aspiring engineers!

*Bienvenido a Hugging Face: ¡Tu parque de diversiones de IA amigable!*

---

## Who We Are: The Cuddle Connoisseurs of AI  
At Hugging Face, we believe in democratizing good machine learning, one heartwarming commit at a time! 💖 We’re a passionate team of 223 cheerful techies, dedicated to creating an AI community where ideas flow as freely as a chocolate fountain at a birthday party. 

*En Hugging Face, creemos en democratizar el buen aprendizaje automático, ¡un conmovedor commit a la vez!💖 Somos un equipo apasionado de 223 entusiastas tecnológicos, dedicados a crear una comunidad de IA donde las ideas fluyen tan libremente como una fuente de chocolate en una fiesta de cumpleaños.*

---

## Our Customers: The Who's Who of AI  
Join over 50,000 organizations ranging from the scholar to the corporate giants, like Google, Microsoft, and even the misunderstood underdog, Grammarly! Remember, whether you’re crafting a language model or chasing after a scary algorithm, we’ve got your back – like a supportive wingman at an awkward first date! 

*Únete a más de 50,000 organizaciones que van desde académicos hasta gigantes corporativos, como Google, Microsoft e incluso el incomprendido perdedor, Grammarly. ¡Recuerda, ya sea que estés creando un modelo de lenguaje o persiguiendo un aterrador algoritmo, nosotros estamos contigo!*

---

## Careers: Join the Hugging Revolution!  
Looking to join a team where the ideas are as sweet as the snacks in the kitchen? Check our job openings! We’re not just about technical skills; we value humor, creativity, and the ability to turn a boring meeting into an improv show! 

*¿Buscas unirte a un equipo donde las ideas son tan dulces como los bocadillos en la cocina? ¡Consulta nuestras ofertas de trabajo! No solo buscamos habilidades técnicas; valoramos el humor, la creatividad y la capacidad de convertir una reunión aburrida en un espectáculo de improvisación.*

---

## Our Awesome Offerings: Better Tools Than a Swiss Army Knife  
We provide tools for all! Whether you're creating models, datasets, or magical Spaces, you can collaborate and build seamlessly. Did we mention we have more models available than your second cousin's collection of cat memes? 😸

*¡Ofrecemos herramientas para todos! Ya sea que estés creando modelos, conjuntos de datos o espacios mágicos, puedes colaborar y construir sin problemas. ¿Mencionamos que tenemos más modelos disponibles que la colección de memes de gatos de tu primo segundo? 😸*

---

## Our Humor: Why did the AI go to therapy?  
Because it had too many layers and couldn’t “unpack” its feelings! 

*¿Por qué fue la IA a terapia? Porque tenía demasiadas capas y no podía “desempacar” sus sentimientos!*

---

So whether you're aiming for a deep learning model that can perform miracles or just want a place to connect with like-minded folks, Hugging Face is more than just a name – it’s a community committed to innovation and support, wrapped in a warm fuzzy feel. Come join us, we’ve got plenty of hugs to go around! 

*Así que, ya sea que busques un modelo de aprendizaje profundo que realice milagros o solo quieras un lugar para conectar con personas afines, Hugging Face es más que un nombre: es una comunidad comprometida con la innovación y el apoyo, envuelta en una cálida sensación de ternura. ¡Ven a unirte a nosotros, tenemos muchos abrazos para repartir!*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>